# Modelo y Simulación de Sistemas I - Proyecto Sustituto
Jimmy White Gómez Ramos 1011590254 \
Levis Javier Aguiar Torres 1005664227\
\
Universidad de Antioquia 2025-2



# Predicción de duración de viajes en taxi - Competición Kaggle

Este notebook implementa un flujo completo para la competición
**NYC Taxi Trip Duration** en Kaggle.
www.kaggle.com/competitions/nyc-taxi-trip-duration/

## Índice de secciones
1. [Conexión con KaggleHub](#1-conexión-con-kagglehub)  
2. [Descarga del dataset de Kaggle](#2-descarga-del-dataset-de-kaggle)  
3. [Importación de librerías y exploración de archivos descargados](#3-importación-de-librerías-y-exploración-de-archivos-descargados)  
4. [Carga de los datos de entrenamiento y prueba](#4-carga-de-los-datos-de-entrenamiento-y-prueba)  
5. [Descripción inicial de los datos](#5-descripción-inicial-de-los-datos)  
6. [Función para calcular la distancia Haversine](#6-función-para-calcular-la-distancia-haversine)  
7. [Función de cálculo de distancia](#7-función-de-cálculo-de-distancia)  
8. [Instalación e importación de librerías meteorológicas](#8-instalación-e-importación-de-librerías-meteorológicas)  
9. [Preprocesamiento de datos climáticos](#9-preprocesamiento-de-datos-climáticos)  
10. [Función de extracción de características temporales](#10-función-de-extracción-de-características-temporales)  
11. [Función para unir clima con datos de viajes](#11-función-para-unir-clima-con-datos-de-viajes)  
12. [Selección de características](#12-selección-de-características)  
13. [Función para dividir los datos en X (features) e y (variable objetivo)](#13-función-para-dividir-los-datos-en-x-features-e-y-variable-objetivo)  
14. [Importación de modelos y utilidades de scikit-learn](#14-importación-de-modelos-y-utilidades-de-scikit-learn)  
15. [Preparación de los datos de entrenamiento y prueba](#15-preparación-de-los-datos-de-entrenamiento-y-prueba)  
16. [Entrenamiento del modelo con LightGBM](#16-entrenamiento-del-modelo-con-lightgbm)  
17. [Predicción y generación de archivo de submission](#17-predicción-y-generación-de-archivo-de-submission)  


### 1. Conexión con KaggleHub  
Se realiza el inicio de sesión en KaggleHub para poder descargar los datos del dataset
de la competición *NYC Taxi Trip Duration*.

In [1]:
import kagglehub
kagglehub.login()

ModuleNotFoundError: No module named 'kagglehub'

### 2. Descarga del dataset de Kaggle
Se descarga el dataset de la competición `nyc-taxi-trip-duration` y se muestra la ruta
donde quedan almacenados los archivos.
**NOTA:** Hay que inscribirse en la competición para poder hacer la descarga remota del dataset.

In [ ]:
nyc_taxi_trip_duration_path = kagglehub.competition_download('nyc-taxi-trip-duration')

print("Ruta de descarga:", nyc_taxi_trip_duration_path)
print('Data source import complete.')

Ruta de descarga: /root/.cache/kagglehub/competitions/nyc-taxi-trip-duration
Data source import complete.


### 3. Importación de librerías y exploración de archivos descargados
En esta celda se importan las librerías principales necesarias para el análisis:

- `numpy`: operaciones matemáticas y algebra lineal.  
- `pandas`: manipulación y análisis de datos en formato tabular.  
- `zipfile`: manejo de archivos comprimidos en formato `.zip`.  
- `math`: funciones matemáticas adicionales.  
- `os`: permite interactuar con el sistema de archivos.  

Posteriormente, se recorre el directorio donde KaggleHub descargó los datos de la competición
(`nyc_taxi_trip_duration_path`) y se imprime la ruta de cada archivo disponible,
para verificar que la descarga fue correcta.


In [ ]:
import numpy as np
import pandas as pd
import zipfile
import math

import os
for dirname, _, filenames in os.walk(nyc_taxi_trip_duration_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/root/.cache/kagglehub/competitions/nyc-taxi-trip-duration/train.zip
/root/.cache/kagglehub/competitions/nyc-taxi-trip-duration/test.zip
/root/.cache/kagglehub/competitions/nyc-taxi-trip-duration/sample_submission.zip


### 4. Carga de los datos de entrenamiento y prueba
Se leen los archivos `train.csv` y `test.csv` que vienen comprimidos en formato `.zip`.  
Las columnas de fechas se parsean como objetos datetime para poder realizar análisis temporales.


In [ ]:
zf = zipfile.ZipFile(f'{nyc_taxi_trip_duration_path}/train.zip')
train = pd.read_csv(zf.open('train.csv'),parse_dates=['pickup_datetime','dropoff_datetime'])

zf = zipfile.ZipFile(f'{nyc_taxi_trip_duration_path}/test.zip')
test = pd.read_csv(zf.open('test.csv'),parse_dates=['pickup_datetime'])

### 5. Descripción inicial de los datos
Se muestran información básica de los datasets de entrenamiento y prueba:
- Dimensiones de las tablas (`shape`)  
- Estadísticos descriptivos (`describe`)  
- Presencia de valores nulos (`isna`)  

Esto permite verificar la calidad inicial y consistencia de los datos.


In [ ]:
print(f'train shape\n{train.shape}')
print(f'train describe\n{train.describe()}')
print(f'train info\n{train.isna().any()}')
print(f'\ntest shape\n{test.shape}')
print(f'\ntest describe\n{test.describe()}')
print(f'test info\n{test.isna().any()}')

train shape
(1458644, 11)
train describe
          vendor_id                pickup_datetime  \
count  1.458644e+06                        1458644   
mean   1.534950e+00  2016-04-01 10:10:24.940037120   
min    1.000000e+00            2016-01-01 00:00:17   
25%    1.000000e+00  2016-02-17 16:46:04.249999872   
50%    2.000000e+00            2016-04-01 17:19:40   
75%    2.000000e+00  2016-05-15 03:56:08.750000128   
max    2.000000e+00            2016-06-30 23:59:39   
std    4.987772e-01                            NaN   

                    dropoff_datetime  passenger_count  pickup_longitude  \
count                        1458644     1.458644e+06      1.458644e+06   
mean   2016-04-01 10:26:24.432310528     1.664530e+00     -7.397349e+01   
min              2016-01-01 00:03:31     0.000000e+00     -1.219333e+02   
25%       2016-02-17 17:05:32.500000     1.000000e+00     -7.399187e+01   
50%              2016-04-01 17:35:12     1.000000e+00     -7.398174e+01   
75%    2016-05-15 04:1

### 6. Función para calcular la distancia Haversine
Se implementa una función auxiliar que calcula la distancia (en km) entre dos puntos
geográficos (latitud y longitud) usando la fórmula de Haversine.  
Esta será la base para generar la variable `distance_km`.


In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calcula la distancia entre dos puntos geográficos utilizando la fórmula de Haversine.

    Args:
        lat1 (float or np.array): Latitud del primer punto en grados decimales.
        lon1 (float or np.array): Longitud del primer punto en grados decimales.
        lat2 (float or np.array): Latitud del segundo punto en grados decimales.
        lon2 (float or np.array): Longitud del segundo punto en grados decimales.

    Returns:
        float or np.array: Distancia entre los dos puntos en kilómetros.
    """
    R = 6371.0

    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance

### 7. Función de cálculo de distancia
Usando la fórmula de Haversine, se agrega al DataFrame una nueva columna `distance_km`
con la distancia calculada entre origen y destino de cada viaje.


In [ ]:
def distance(df):
    """
    Agrega al DataFrame una nueva columna con la distancia recorrida en cada viaje,
    calculada a partir de las coordenadas de origen y destino.

    Args:
        df (pd.DataFrame): DataFrame que contiene las columnas
            'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'.

    Returns:
        pd.DataFrame: DataFrame original con una nueva columna 'distance_km'.
    """
    coords  = (df['pickup_latitude'],df['pickup_longitude'],df['dropoff_latitude'],df['dropoff_longitude'])
    df['distance_km'] = haversine(*coords)
    # df['speed_m_s'] = df['distance_km']*1000/df['trip_duration']
    # df['diff_seconds'] = (df['dropoff_datetime'] - df['pickup_datetime']).dt.total_seconds()
    return df

### 8. Instalación e importación de librerías meteorológicas
Se instala `meteostat`, se importa la clase `Point` y los métodos `Daily` y `Hourly`
para obtener datos climáticos históricos de Nueva York en el mismo rango de fechas del dataset.


In [ ]:
%pip install meteostat
from meteostat import Point, Daily, Hourly
from datetime import datetime

location = Point(40.7128, -74.0060)

start = datetime(2015, 12, 31)
end = datetime(2016, 7, 31)

data_daily = Daily(location, start, end).fetch()
data_hourly = Hourly(location, start, end).fetch()


### 9. Preprocesamiento de datos climáticos
Se reinicia el índice, se ajustan zonas horarias y se convierte la columna de tiempo
al huso horario de Nueva York para poder unirlo luego con los datos de taxis.


In [ ]:
data_hourly = data_hourly.reset_index().rename(columns={'DatetimeIndex': 'weather_datetime'})
data_hourly['time'] = data_hourly['time'].dt.tz_localize('UTC')
data_hourly['time_ny'] = data_hourly['time'].dt.tz_convert('America/New_York')
data_hourly['time_ny'] = data_hourly['time_ny'].dt.tz_localize(None)

### 10. Función de extracción de características temporales
Genera variables adicionales relacionadas con la fecha y hora del viaje, como:
día, mes, año, hora, semana y día de la semana.


In [ ]:
def time_features(df):
    """
    Extrae características temporales de la columna 'pickup_datetime' para enriquecer el dataset.

    Args:
        df (pd.DataFrame): DataFrame que contiene la columna 'pickup_datetime'.

    Returns:
        pd.DataFrame: DataFrame con nuevas columnas:
            - 'pickup_day': día del mes
            - 'pickup_month': mes
            - 'pickup_year': año
            - 'pickup_hour': hora del día
            - 'pickup_week': número de semana
            - 'pickup_datetime_hour_trunc': fecha truncada a la hora
            - 'pickup_dayofweek': día de la semana (0 = lunes, 6 = domingo)
    """
    df['pickup_day'] = df['pickup_datetime'].dt.day
    df['pickup_month'] = df['pickup_datetime'].dt.month
    df['pickup_year'] = df['pickup_datetime'].dt.year
    df['pickup_hour'] = df['pickup_datetime'].dt.hour
    df['pickup_week'] = df['pickup_datetime'].dt.isocalendar()['week']
    df['pickup_datetime_hour_trunc'] = df['pickup_datetime'].dt.floor('h')
    df['pickup_dayofweek'] = df['pickup_datetime'].dt.dayofweek
    return df

### 11. Función para unir clima con datos de viajes
Realiza un `merge` entre el DataFrame de taxis y el de clima, usando como clave la hora
truncada del `pickup_datetime`.


In [ ]:
def weather_merge(df):
    """
    Une los datos meteorológicos horarios con el dataset de viajes de taxi.

    Args:
        df (pd.DataFrame): DataFrame con la columna 'pickup_datetime_hour_trunc'
            que representa la fecha-hora truncada al inicio de la hora.

    Returns:
        pd.DataFrame: DataFrame con las columnas originales más las variables
        climáticas provenientes del DataFrame 'data_hourly' (ej. temperatura, precipitación).
    """
    merged_df = df.merge(data_hourly,how='left',left_on='pickup_datetime_hour_trunc',right_on='time_ny')
    return merged_df

### 12. Selección de características
Se define la lista de variables (`features`) que se usarán como entrada en el modelo de predicción.


In [ ]:
features = [
    'passenger_count',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'distance_km',
    'pickup_day',
    # 'pickup_month',
    'pickup_hour',
    # 'pickup_year',
    # 'pickup_week',
    'pickup_dayofweek',
    'temp','prcp'
]

### 13. Función para dividir los datos en X (features) e y (variable objetivo)
Se separa la variable `trip_duration` de las variables predictoras seleccionadas.


In [ ]:
def train_split(df):
    """
    Separa las variables predictoras (X) y la variable objetivo (y) del dataset.

    Args:
        df (pd.DataFrame): DataFrame que contiene las columnas de entrada definidas en 'features'
            y la columna objetivo 'trip_duration'.

    Returns:
        tuple:
            - X_train (pd.DataFrame): DataFrame con las variables predictoras seleccionadas.
            - y_train (pd.Series): Serie con la variable objetivo (duración del viaje).
    """
    X_train = df[features]
    y_train = df['trip_duration']
    return X_train, y_train

### 14. Importación de modelos y utilidades de scikit-learn
Se importan modelos de regresión y métricas de evaluación.  
Aunque también se consideran Random Forest y Gradient Boosting, en esta versión se empleará LightGBM.


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

### 15. Preparación de los datos de entrenamiento y prueba
Se aplican las funciones de cálculo de distancia, extracción de variables temporales
y unión con el clima para preparar los datasets antes de entrenar el modelo.


In [ ]:
train_df = distance(train)
train_df = time_features(train_df)
train_merged = weather_merge(train_df)
X_train, y_train = train_split(train_merged)

test_df = distance(test)
test_df = time_features(test_df)
test_merged = weather_merge(test_df)


### 16. Entrenamiento del modelo con LightGBM
Se entrena un modelo de regresión `LGBMRegressor` utilizando los datos procesados.


In [ ]:
import lightgbm as lgb
from lightgbm import LGBMRegressor

lgbm = lgb.LGBMRegressor()
lgbm.fit(X_train, y_train)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.185501 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1500
[LightGBM] [Info] Number of data points in the train set: 1458644, number of used features: 11
[LightGBM] [Info] Start training from score 959.492273


LGBMRegressor()

### 17. Predicción y generación de archivo de submission
Se generan las predicciones para el conjunto de prueba y se guardan en un archivo
`submission.csv` con el formato requerido por la competición.  
Este archivo está listo para ser cargado en Kaggle como solución.


In [ ]:
y_pred = lgbm.predict(test_merged[features])

In [ ]:
submission = pd.DataFrame({'id': test.id, 'trip_duration': (y_pred)})

In [ ]:
submission.to_csv("submission.csv", index=False)